## Setting up Undamped Unforced Duffing equation

In [1]:
from ptlpinns.perturbation import LPM
from ptlpinns.odes import equations, numerical
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

In [2]:
w_0 = 1
t_span = (0, 15)
N = 5120
t_eval = np.linspace(t_span[0], t_span[1], N)
ic = np.array([1, 0])
epsilon = 0.6

## Undamped

In [3]:
def forcing_cos(t):
    return np.cos(t)

def forcing_zero(t):
    return np.zeroes_like(t)

def ode_nl_duffing(t, y):
    return np.array([y[1], - (w_0 ** 2) * y[0] - epsilon * (y[0] ** 3) + forcing_zero(t)])

In [4]:
ode_0 = equations.ode_oscillator_1D(w_0=1, zeta=0, forcing_1D=lambda t: np.zeros_like(t), epsilon=0)
x_0 = numerical.solve_ode_equation(ode_0, (t_eval[0], t_eval[-1]), t_eval, [1,0])

In [5]:
x_lin = [x_0[0, :]]
ddot_x_lin = [np.gradient(x_0[1, :], t_eval)]
w_sol = [w_0]

In [6]:
n_corrections = 6

for p in range(1, n_corrections + 1):

    print(f"calculating term p: {p}")
    w_n = LPM.calc_w_n(w_list=w_sol, x=x_lin, x_ddot=ddot_x_lin, t=t_eval)
    print(f"omega {p}: {w_n}")

    x_n_forcing = LPM.calculate_forcing(w_n=w_n, w_list=w_sol, x=x_lin, x_ddot=ddot_x_lin)

    # defining forcing array as a function
    x_n_forcing_interp = CubicSpline(t_eval, x_n_forcing)

    # defining ode for x_n
    def ode_x_n(t, y):
        return np.array([y[1], - (w_0 ** 2) * y[0] + x_n_forcing_interp(t)])
    
    # solving x_n ode
    x_n_lin = (numerical.solve_ode_equation(ode_x_n, t_span, t_eval, np.array([0,0])))[0]   

    w_sol.append(w_n)
    x_lin.append(x_n_lin)
    ddot_x_lin.append(np.gradient(np.gradient(x_n_lin, t_eval), t_eval))

calculating term p: 1
omega 1: 0.37497234957900066
calculating term p: 2
omega 2: -0.08197277132457766
calculating term p: 3
omega 3: 0.03949683720534082
calculating term p: 4
omega 4: -0.024931950862469274
calculating term p: 5
omega 5: 0.0179466743295856
calculating term p: 6
omega 6: -0.01390478064464109


In [8]:
LPM.calculate_w_series(w_sol, epsilon, rwtol=1e-12)


[1,
 np.float64(1.2249834097474004),
 np.float64(1.1954732120705525),
 np.float64(1.2040045289069061),
 np.float64(1.2007733480751301),
 np.float64(1.2021688814709988),
 np.float64(1.2015201400252424)]

In [7]:
nl_numerical_duffing = numerical.solve_ode_equation(ode_nl_duffing, t_span, t_eval, ic)

AttributeError: module 'numpy' has no attribute 'zeroes_like'